In [1]:
!pip install pyaudio
!pip install wave
!pip install numpy
!pip install faster-whisper
!pip install keyboard


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pyaudio
import wave
import os
import numpy as np
from faster_whisper import WhisperModel
import keyboard
import threading

c:\projects\Personal\notebook-scripts\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Global variables
is_recording = False
frames = []

In [4]:
# Load the Faster Whisper model
model_size = "base"

# Run on GPU with FP16
model = WhisperModel(model_size, device="cuda", compute_type="float16")

In [5]:
def record_audio(filename, sample_rate=16000):
    """Record audio from the microphone and save it to a file."""
    global is_recording, frames
    
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=sample_rate,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("Press SPACE to start recording...")
    keyboard.wait('space')
    print("Recording... Press SPACE again to stop.")
    
    is_recording = True
    frames = []

    while is_recording:
        data = stream.read(CHUNK)
        frames.append(data)

    print("Recording finished.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()

In [6]:
def transcribe_audio(filename):
    """Transcribe the audio file using Faster Whisper."""
    segments, info = model.transcribe(filename, beam_size=5)
    return " ".join([segment.text for segment in segments])

In [7]:
def on_press_space():
    global is_recording
    if is_recording:
        is_recording = False
        
# Set up keyboard listener
keyboard.on_press_key('space', lambda _: on_press_space())

<function keyboard.hook_key.<locals>.remove_()>

In [8]:
# Record audio
audio_filename = "audio.wav"
record_thread = threading.Thread(target=record_audio, args=(audio_filename,))
record_thread.start()
record_thread.join()

Press SPACE to start recording...
Recording... Press SPACE again to stop.
Recording finished.


In [9]:
# Transcribe the recorded audio
transcription = transcribe_audio(audio_filename)
print("Transcription:", transcription)

Transcription:  Alright, so this is another test. It seems to work fine, but I want to see if it's correctly doing it.


In [10]:
# Clean up
keyboard.unhook_all()

In [11]:
# Check if the output file exists and delete it if it does
if os.path.exists(audio_filename):
    os.remove(audio_filename)